In [1]:
import numpy as np
#import tensorflow as tf

## Tetris engine

In [106]:
class Tetris():
    
    def __init__(self, width, height):
        
        self.width = width
        self.height = height
        self.grid = np.append(np.zeros(width*height), -np.ones(width))
        self.score = 0
        
        self.current_tetromino = None
        self.current_position = None
        self.current_rotation = None
        
        lTetromino = [
                [1,width+1,width*2+1,2],
                [width, width+1,width+2,width*2+2],
                [1,width+1,width*2+1,width*2],
                [0,width, width+1,width+2]
            ]
    
        zTetromino = [
                [1,2,width,width+1],
                [0,width,width+1,width*2+1],
                [1,2,width,width+1],
                [0,width,width+1,width*2+1]
            ]

        tTetromino = [
                [1,width,width+1,width+2],
                [1,width+1,width+2,width*2+1],
                [width,width+1,width+2,width*2+1],
                [1,width,width+1,width*2+1]
            ]

        oTetromino = [
                [0,1,width,width+1],
                [0,1,width,width+1],
                [0,1,width,width+1],
                [0,1,width,width+1]
            ]

        iTetromino = [
                [1,width+1,width*2+1,width*3+1],
                [width,width+1,width+2,width+3],
                [1,width+1,width*2+1,width*3+1],
                [width,width+1,width+2,width+3]
            ]

        self.theTetrominoes = [lTetromino, zTetromino, tTetromino, oTetromino,iTetromino]
        
        self.theTetrominoSizes = np.max(np.array(game.theTetrominoes) % game.width, axis=(1,2))+1
        
        self.status = "ONGOING"
        
    
    def start_new_tetromino(self):
        
        self.current_tetromino = np.random.choice(range(len(self.theTetrominoes)))
        self.current_position = 4
        self.current_rotation = 0
        
        self.print_game()
    
    
    def print_game(self):
        
        grid = self.grid.copy()
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        for pos in tetromino:
            grid[pos + self.current_position] = 2
        
        print( np.reshape(grid,(self.height+1,self.width)) )
        
        
    def move(self, action):
        
        reward = 0
        
        if self.check_tetromino_stop():
            self.start_new_tetromino()
            
        if action==0:
            # do nothing
            print("nothing")
        elif action==1:
            # rotate
            print("rotate")
            self.current_rotation, reward = self.rotate()
        elif action==2:
            # go left
            print("left")
            self.current_position, reward = self.move_left()
        elif action==3:
            # go right
            print("right")
            self.current_position, reward = self.move_right()
        elif action==4:
            # go down
            print("down")
            self.current_position += self.width
        else:
            print("Invalid action")
        
        if self.check_tetromino_stop():
            self.start_new_tetromino()
        else:
            self.current_position += self.width
            if self.check_tetromino_stop():
                self.start_new_tetromino()

        reward += self.check_lines_and_update_score()
        
        print(f"Reward: {reward}")

    def rotate(self):
        
        #tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        rotation = (self.current_rotation + 1) %4
        rotated_tetromino = self.theTetrominoes[self.current_tetromino][rotation]
        width = self.width
        
        tetromino_size = self.theTetrominoSizes[self.current_tetromino]
        
        for pos in rotated_tetromino:
            if self.grid[pos+self.current_position]==1:
                return self.current_rotation, -10
        
        if tetromino_size==3:
            if (self.current_position+1-1) % width==0 or (self.current_position+1+1) % width==0:
                return self.current_rotation, -10
        if tetromino_size==4 and self.current_rotation in [0,2]:
            if (self.current_position+1-1) % width==0 or (self.current_position+2+1) % width==0:
                return self.current_rotation, -10
        
        return rotation, 0
        
        
    def move_left(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        position = self.current_position - 1
        width = self.width
        
        for pos in tetromino:
            if (pos+self.current_position)//width!=(pos+position)//width or self.grid[pos+position]==1:
                #print(self.current_position, position, pos)
                self.print_game()
                return self.current_position, -10
        
        return position, 0
    
    def move_right(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        position = self.current_position + 1
        width = self.width
        
        for pos in tetromino:
            if (pos+self.current_position)//width!=(pos+position)//width or self.grid[pos+position]==1:
                #print(self.current_position, position, pos)
                self.print_game()
                return self.current_position, -10
        
        return position, 0
    
    def check_tetromino_stop(self):
        
        tetromino = self.theTetrominoes[self.current_tetromino][self.current_rotation]
        for pos in tetromino:
            if self.grid[pos + self.current_position + self.width] in [-1, 1]:
                for final_pos in tetromino:
                    self.grid[final_pos + self.current_position] = 1
                return True
        
        return False
    
    def check_lines_and_update_score(self):
        
        for h in range(self.height):
            
            if (self.grid[h*self.width:(h+1)*self.width]==1).all():
                self.grid = np.delete(self.grid, range(h*self.width,(h+1)*self.width))
                self.score += 1
                self.grid = np.insert( self.grid, 0, np.zeros(self.width))
                print(f"{h} line removed score = {self.score}")
                return 20
            
        if (self.grid[:self.width]==1).any():
            self.status = "OVER"
            print("Game Over")
            return -20
    
        return 0
            

## Testing the tetromino for the rotation

In [107]:
game = Tetris(width = 10, height = 20)

In [108]:
sizes = np.max(np.array(game.theTetrominoes) % game.width, axis=(1,2)) +1
print(sizes)

[3 3 3 2 4]


## Testing the game

In [109]:
game = Tetris(width = 10, height = 20)
game.start_new_tetromino()

while game.status!="OVER":
    game.move(np.random.choice(range(5)))
game.print_game()

[[ 0.  0.  0.  0.  0.  2.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  2.  2.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
left
Reward: 0
nothing
Reward: 0
left
Reward: 0
left
Reward: 0
right
Reward: 0
right
Reward: 0
r